# BERT의 문장 임베딩(SBERT)을 이용한 한국어 챗봇
- 참고: \
https://wikidocs.net/154530 \
https://github.com/kairess/mental-health-chatbot

In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 38.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=1359f61b5ea583ab1d71257bc444157e1ce8221c1dd772bc45e24813dc3e990a
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [2]:
import time
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

In [3]:
#https://huggingface.co/models?library=sentence-transformers&sort=downloads&search=ko

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
print(train_data.count())
train_data.head()

Q        11823
A        11823
label    11823
dtype: int64


,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [5]:
train_data.loc[0, 'Q']
print(model.encode(train_data.loc[0, 'Q'])[:100])

[ 0.20179531 -0.03443793  1.5395737   0.01069728  0.3797922   0.14249353
  0.18286432  0.5583127  -0.3948895  -0.11765724 -0.23298924  0.0339581
 -0.24710451  0.31453586  0.56095064 -0.10551719 -0.2928853  -0.20406021
  0.03351972 -0.3213897  -0.18638428  0.08559985  0.15107217 -0.38457727
 -0.29281446 -0.43999526  0.48339477 -0.96764404 -0.01514992  0.00377249
  0.24931629 -0.06040543  0.2290055  -0.02450772 -0.04226572 -0.07887703
 -0.5185486  -0.02258337 -0.11383002 -0.47592595  0.80632675 -0.03979286
  0.38371733  0.22324897 -0.0762537  -0.07432782  0.03572281  0.25903684
 -0.34493852 -0.5932294  -0.8210689  -0.40504244  0.11665888  0.2762489
 -0.27738914 -0.11598583 -0.01691463 -0.25258973  0.19903351  0.6454387
  0.39483887 -0.5658856   0.34821156  0.6537091  -0.15174352  0.17747317
  0.31699148 -0.03903285 -0.20392574  0.44761378  0.7113443   0.16152243
  0.16321644  0.6009315  -0.4119127   0.06427157  0.17610325  0.27279142
 -0.17184258  0.55049235  0.08984318 -0.07437636  0.70

In [6]:
%%time
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

CPU times: user 2min 34s, sys: 446 ms, total: 2min 34s
Wall time: 2min 37s


In [7]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [8]:
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)

    return "{0:0.4f}".format(train_data['score'][0]), train_data.loc[train_data['score'].idxmax()]['A']

In [9]:
ans1=return_answer('챗봇 만들기 너무 어려워')
print(f'답변: {ans1[1]} / 정확도: {ans1[0]}')

답변: 신고하고 차단해보세요. / 정확도: 0.3991


In [10]:
while True:
  user_input=input('▶ 당신: ')
  if user_input in ['X', 'x']:
    print('※ System: 종료합니다')
    break
  bot_ans=return_answer(user_input)
  print('▷ 챗봇:', bot_ans[1])
  print('정확도:', bot_ans[0], end='\n\n')

▶ 당신: 반가워
▷ 챗봇: 저도 반가워요.
정확도: 0.8157

▶ 당신: 배고파
▷ 챗봇: 얼른 맛난 음식 드세요.
정확도: 0.2564

▶ 당신: 뭐하면 좋을까
▷ 챗봇: 혼자 남겨진 기분은 감히 상상도 못해요.
정확도: 0.8053

▶ 당신: 여친이랑 헤어졌어
▷ 챗봇: 허전한 마음이 들겠어요.
정확도: 0.8307

▶ 당신: 너무 우울해
▷ 챗봇: 많이 힘들었죠.
정확도: 0.7785

▶ 당신: 병원 가는 건 어떨까
▷ 챗봇: 꼬박꼬박 챙겨 드세요.
정확도: 0.7695

▶ 당신: 어행가면 기분이 풀릴 것 같아
▷ 챗봇: 그렇게 잊어가는 건가봐요.
정확도: 0.6097

▶ 당신: x
※ System: 종료합니다
